In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from FRVRS import fu, nu
import numpy as np
import os
import os.path as osp
import pandas as pd
import re


# Build and deidentify the dataset of OSU Simulation Session Logs

In [3]:

# Get whether we can bring the dataset up from storage
pickle_loaded = False
# if nu.pickle_exists('frvrs_logs_df'):
#     try:
#         frvrs_logs_df = nu.load_object('frvrs_logs_df')
#         pickle_loaded = True
#     except AttributeError as e: print(str(e).strip())
# if (not pickle_loaded) and nu.csv_exists('frvrs_logs_df', folder_path=nu.saves_folder):
#     frvrs_logs_df = nu.load_csv(csv_name='frvrs_logs_df', folder_path=nu.saves_folder)
#     pickle_loaded = True
pickle_loaded

False

In [4]:

# Get all logs into one data frame
if not pickle_loaded:
    
    # Add the CSVs to the data frame
    frvrs_logs_df = fu.concatonate_logs()
    
    # Remove numerically-named columns
    columns_list = [x for x in frvrs_logs_df.columns if not re.search(r'\d+', str(x))]
    frvrs_logs_df = frvrs_logs_df[columns_list]
    
    # Convert 'TRUE' and 'FALSE' to boolean values
    for cn in fu.boolean_columns_list: frvrs_logs_df[cn] = frvrs_logs_df[cn].map({'TRUE': True, 'FALSE': False, 'True': True, 'False': False})
    
    # Convert the nulls into NaNs
    for cn in frvrs_logs_df.columns: frvrs_logs_df[cn] = frvrs_logs_df[cn].replace('null', np.nan)
    
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    nu.save_data_frames(frvrs_logs_df=frvrs_logs_df)
    pickle_loaded = True
print(frvrs_logs_df.shape) # (829116, 106)
search_regex = re.compile(r'^Applied[A-Z]')
columns_list = nu.get_regexed_columns(frvrs_logs_df, search_regex=search_regex)
print(columns_list)
df = nu.get_regexed_dataframe(frvrs_logs_df, columns_list, search_regex=search_regex)
display(df.sample(min(4, df.shape[0])).dropna(axis='columns', how='all').T)

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv
(829116, 106)
['tool_applied_attach_message']


,51
action_type,TOOL_APPLIED
action_tick,127628
event_time,2023-03-10 07:55:43
session_uuid,cf714cd2-2824-4ad2-8d98-3199b1df0bdb
file_name,All CSV files renamed by date/03.10.23.0753.csv
logger_version,1.0
tool_applied_type,Needle
tool_applied_attachment_point,RChest_Breathe1 (UnityEngine.GameObject)
tool_applied_tool_location,cath_needle (UnityEngine.GameObject)
tool_applied_data,right_chest



## Check for duplicate file ingestion

In [5]:

# Filter all the rows that have more than one unique value in the file_name column for each value in the session_uuid column
mask_series = (frvrs_logs_df.groupby('session_uuid').file_name.transform(pd.Series.nunique) > 1)
assert frvrs_logs_df[mask_series].shape[0] == 0, "You have duplicate files"
# columns_list = ['session_uuid', 'file_name']
# for (session_uuid, file_name), df in frvrs_logs_df[mask_series][columns_list].drop_duplicates().sort_values(columns_list).groupby(columns_list):
#     if not file_name.startswith('Double runs removed/'):
#         file_path = osp.join(fu.data_logs_folder, *file_name.split('/'))
#         os.remove(file_path)


## Add new features according to your increasing domain knowledge


### Modalize separate columns into one

In [6]:

# Modalize into one patient ID column if possible
new_column_name = 'patient_id'
if (new_column_name not in frvrs_logs_df.columns):
    frvrs_logs_df = nu.modalize_columns(frvrs_logs_df, fu.patient_id_columns_list, new_column_name)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    nu.save_data_frames(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (829116, 108)
display(frvrs_logs_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}).sort_values('record_count', ascending=False).head(5))

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv
(829116, 107)


,record_count
patient_id,
Helga_10 Root,191293
Gloria_8 Root,170979
Bob_9 Root,30678
Mike_2 Root,17804
Mike_0 Root,11035


In [7]:

# Modalize into one injury ID column if possible
new_column_name = 'injury_id'
if (new_column_name not in frvrs_logs_df.columns):
    frvrs_logs_df = nu.modalize_columns(frvrs_logs_df, fu.injury_id_columns_list, new_column_name)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    nu.save_data_frames(frvrs_logs_df=frvrs_logs_df)
    
    print(frvrs_logs_df.shape) # (829116, 109)
display(frvrs_logs_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}).sort_values('record_count', ascending=False).head(5))

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv
(829116, 108)


,record_count
injury_id,
R Chest Collapse,1242
R Shin Amputation,1102
L Thigh Puncture,1018
L Thigh Laceration,888
R Thigh Laceration,875


In [8]:

# Modalize into one location ID column if possible
new_column_name = 'location_id'
if (new_column_name not in frvrs_logs_df.columns):
    frvrs_logs_df = nu.modalize_columns(frvrs_logs_df, fu.location_id_columns_list, new_column_name)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    nu.save_data_frames(frvrs_logs_df=frvrs_logs_df)
    
    print(frvrs_logs_df.shape) # (829116, 108)
display(frvrs_logs_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}).sort_values('record_count', ascending=False).head(5))

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv
(829116, 109)


,record_count
location_id,
"(4.0, 0.0, -4.8)",8213
"(4.1, 0.0, -4.8)",4373
"(3.9, 0.0, -4.9)",4307
"(5.7, 0.0, -5.7)",4083
"(2.7, 0.0, -5.7)",3860


In [9]:

# Modalize into one patient sort column if possible
new_column_name = 'patient_sort'
if (new_column_name not in frvrs_logs_df.columns):
    frvrs_logs_df = nu.modalize_columns(frvrs_logs_df, fu.sort_columns_list, new_column_name)
    frvrs_logs_df[new_column_name] = frvrs_logs_df[new_column_name].astype(fu.sort_category_order)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    nu.save_data_frames(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (829116, 110)

display(frvrs_logs_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv
(829116, 110)


,record_count
patient_sort,
still,5438
waver,8196
walker,4960


In [10]:

# Modalize into one patient pulse column if possible
new_column_name = 'patient_pulse'
if (new_column_name not in frvrs_logs_df.columns):
    frvrs_logs_df = nu.modalize_columns(frvrs_logs_df, fu.pulse_columns_list, new_column_name)
    frvrs_logs_df[new_column_name] = frvrs_logs_df[new_column_name].astype(fu.pulse_category_order)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    nu.save_data_frames(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (829116, 110)
    
display(frvrs_logs_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv
(829116, 111)


,record_count
patient_pulse,
none,2363
faint,2875
fast,8043
normal,5313


In [11]:

# Modalize into one patient salt column if possible
new_column_name = 'patient_salt'
if (new_column_name not in frvrs_logs_df.columns):
    frvrs_logs_df = nu.modalize_columns(frvrs_logs_df, fu.salt_columns_list, new_column_name)
    frvrs_logs_df[new_column_name] = frvrs_logs_df[new_column_name].astype(fu.salt_category_order)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    nu.save_data_frames(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (829116, 113)
    
display(frvrs_logs_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv
(829116, 112)


,record_count
patient_salt,
DEAD,1338
EXPECTANT,1866
IMMEDIATE,7815
DELAYED,3756
MINIMAL,3819


In [12]:

# Modalize into one patient hearing column if possible
new_column_name = 'patient_hearing'
if (new_column_name not in frvrs_logs_df.columns):
    frvrs_logs_df = nu.modalize_columns(frvrs_logs_df, fu.hearing_columns_list, new_column_name)
    frvrs_logs_df[new_column_name] = frvrs_logs_df[new_column_name].astype(fu.hearing_category_order)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    nu.save_data_frames(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (829116, 110)
    
display(frvrs_logs_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv
(829116, 113)


,record_count
patient_hearing,
none,1987
limited,750
normal,10539


In [13]:

# Modalize into one patient breath column if possible
new_column_name = 'patient_breath'
if (new_column_name not in frvrs_logs_df.columns):
    frvrs_logs_df = nu.modalize_columns(frvrs_logs_df, fu.breath_columns_list, new_column_name)
    frvrs_logs_df[new_column_name] = frvrs_logs_df[new_column_name].astype(fu.breath_category_order)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    nu.save_data_frames(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (829116, 110)
    
display(frvrs_logs_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv
(829116, 114)


,record_count
patient_breath,
none,1338
collapsedRight,1937
restricted,2045
fast,1753
normal,11521


In [14]:

# Modalize into one patient mood column if possible
new_column_name = 'patient_mood'
if (new_column_name not in frvrs_logs_df.columns):
    frvrs_logs_df = nu.modalize_columns(frvrs_logs_df, fu.mood_columns_list, new_column_name)
    frvrs_logs_df[new_column_name] = frvrs_logs_df[new_column_name].astype(fu.mood_category_order)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    nu.save_data_frames(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (829116, 110)
    
display(frvrs_logs_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv
(829116, 115)


,record_count
patient_mood,
dead,1375
unresponsive,3863
agony,1508
upset,8841
calm,3007


In [15]:

# Modalize into one patient pose column if possible
new_column_name = 'patient_pose'
if (new_column_name not in frvrs_logs_df.columns):
    frvrs_logs_df = nu.modalize_columns(frvrs_logs_df, fu.pose_columns_list, new_column_name)
    frvrs_logs_df[new_column_name] = frvrs_logs_df[new_column_name].astype(fu.pose_category_order)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    nu.save_data_frames(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (829116, 110)
    
display(frvrs_logs_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv
(829116, 116)


,record_count
patient_pose,
supine,5603
fetal,5046
sittingGround,4561
kneeling,66
recovery,718
standing,2574


In [16]:

# Modalize into one injury severity column if possible
new_column_name = 'injury_severity'
if (new_column_name not in frvrs_logs_df.columns):
    frvrs_logs_df = nu.modalize_columns(frvrs_logs_df, fu.severity_columns_list, new_column_name)
    frvrs_logs_df[new_column_name] = frvrs_logs_df[new_column_name].astype(fu.severity_category_order)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    nu.save_data_frames(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (829116, 120)
    
display(frvrs_logs_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv
(829116, 117)


,record_count
injury_severity,
high,4647
medium,5643
low,2867


In [17]:

# Modalize into one injury required_procedure column if possible
new_column_name = 'injury_required_procedure'
if (new_column_name not in frvrs_logs_df.columns):
    frvrs_logs_df = nu.modalize_columns(frvrs_logs_df, fu.required_procedure_columns_list, new_column_name)
    frvrs_logs_df[new_column_name] = frvrs_logs_df[new_column_name].astype(fu.required_procedure_category_order)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    nu.save_data_frames(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (829116, 110)
    
display(frvrs_logs_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv
(829116, 118)


,record_count
injury_required_procedure,
tourniquet,4547
gauzePressure,2991
decompress,1606
woundpack,2512
airway,703
none,798


In [18]:

# Modalize into one injury body_region column if possible
new_column_name = 'injury_body_region'
if (new_column_name not in frvrs_logs_df.columns):
    frvrs_logs_df = nu.modalize_columns(frvrs_logs_df, fu.body_region_columns_list, new_column_name)
    frvrs_logs_df[new_column_name] = frvrs_logs_df[new_column_name].astype(fu.body_region_category_order)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    nu.save_data_frames(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (829116, 120)
    
display(frvrs_logs_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv
(829116, 119)


,record_count
injury_body_region,
head,1614
neck,18
chest,2162
abdomen,1989
leftLeg,2080
rightLeg,3207
rightArm,829
leftArm,1258


In [19]:

# Modalize into one tool type column if possible
new_column_name = 'tool_type'
if (new_column_name not in frvrs_logs_df.columns):
    frvrs_logs_df = nu.modalize_columns(frvrs_logs_df, fu.tool_type_columns_list, new_column_name)
    frvrs_logs_df[new_column_name] = frvrs_logs_df[new_column_name].astype(fu.tool_type_category_order)
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    nu.save_data_frames(frvrs_logs_df=frvrs_logs_df)
    print(frvrs_logs_df.shape) # (829116, 122)
    
display(frvrs_logs_df.groupby(new_column_name).size().to_frame().rename(columns={0: 'record_count'}))

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv
(829116, 120)


,record_count
tool_type,
Tourniquet,74483
Gauze_Pack,1452
Needle,36848
Naso,10
Nasal Airway,3301
Gauze_Dressing,1445



### Convert text columns to categorical

In [20]:

new_column_name = 'pulse_taken_pulse_name'
if (new_column_name in frvrs_logs_df.columns):
    frvrs_logs_df[new_column_name] = frvrs_logs_df[new_column_name].astype(fu.pulse_name_category_order)
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    nu.save_data_frames(frvrs_logs_df=frvrs_logs_df)

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv


In [21]:

new_column_name = 'tool_applied_data'
if (new_column_name in frvrs_logs_df.columns):
    frvrs_logs_df[new_column_name] = frvrs_logs_df[new_column_name].astype(fu.tool_data_category_order)
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    nu.save_data_frames(frvrs_logs_df=frvrs_logs_df)

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv



## Mask PII

In [22]:

# Mask voice capture PII. OSU screened all of the **VOICE_COMMAND** and **VOICE_CAPTURE** lines and
# replaced any names with either Max or Jane, regardless of whether the name was that of the responder.
# But, just to make sure...
columns_list = ['voice_command_command_description', 'voice_capture_message']
if not frvrs_logs_df[columns_list].applymap(lambda x: '[PERSON]' in str(x), na_action='ignore').sum().sum():
    import spacy
    try: nlp = spacy.load('en_core_web_sm')
    except OSError as e:
        print(str(e).strip())
        command_str = f'{sys.executable} -m spacy download en_core_web_sm --quiet'
        print(command_str)
        !{command_str}
        nlp = spacy.load('en_core_web_sm')
    import en_core_web_sm
    nlp = en_core_web_sm.load()
    
    mask_series = frvrs_logs_df.voice_command_command_description.isnull() & frvrs_logs_df.voice_capture_message.isnull()
    df = frvrs_logs_df[~mask_series]
    def mask_pii(srs):
        for idx in columns_list:
            new_text = srs[idx]
            if pd.notnull(new_text):
                doc = nlp(new_text)
                for entity in doc.ents:
                    if entity.label_ == 'PERSON': new_text = re.sub('\\b' + entity.text + '\\b', '[PERSON]', new_text)
                srs[idx] = new_text
    
        return srs
    
    for row_index, row_series in df.apply(mask_pii, axis='columns')[columns_list].iterrows():
        for column_name, column_value in row_series.items():
            if pd.notnull(column_value): frvrs_logs_df.loc[row_index, column_name] = column_value
    
    # Store the results and show the new data frame shape
    nu.store_objects(frvrs_logs_df=frvrs_logs_df)
    nu.save_data_frames(frvrs_logs_df=frvrs_logs_df)
    
    print(frvrs_logs_df.shape) # (829116, 113)

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/frvrs_logs_df.csv
(829116, 120)
